# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao gabriel da cachoeira,-0.1303,-67.0892,146.994444,97,62,0.25,BR,1720006232
1,1,broken hill,-31.9500,141.4333,139.461111,69,41,4.97,AU,1720006234
2,2,dakhla,23.6848,-15.9580,146.200000,73,62,11.32,EH,1720006237
3,3,vorgashor,67.5833,63.9500,142.150000,90,100,6.38,RU,1720006239
4,4,kavaratti,10.5669,72.6420,149.727778,85,100,6.91,IN,1720006241


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity (%)",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_wc_City = city_data_df.loc[(city_data_df["Wind Speed"] <= 3) & (city_data_df["Cloudiness"] == 0) & \
                                   (city_data_df["Max Temp"] >= 110) & (city_data_df["Max Temp"] <= 145)].dropna()


# Drop any rows with null values
ideal_wc_City=ideal_wc_City.dropna()
ideal_wc_City.reset_index(drop=True)

# Display sample data
ideal_wc_City.head()
len(ideal_wc_City)

13

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_wc_City.copy(deep=True)
hotel_df = hotel_df.loc[:,["City","Country", "Lat", "Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,bethel,US,41.3712,-73.4140,88,
28,smithers,CA,54.7804,-127.1743,81,
62,east london,ZA,-33.0153,27.9116,59,
94,brookings,US,44.3114,-96.7984,91,
130,colonia del sacramento,UY,-34.4667,-57.8500,70,
133,alice springs,AU,-23.7000,133.8833,81,
175,susanville,US,40.4163,-120.6530,36,
210,talcahuano,CL,-36.7167,-73.1167,93,
220,fortuna,US,40.5982,-124.1573,100,
330,bayside,US,40.8424,-124.0637,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
smithers - nearest hotel: Sunshine Inn Hotel
east london - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
colonia del sacramento - nearest hotel: Radisson Hotel Colonia del Sacramento
alice springs - nearest hotel: Aurora Alice Springs
susanville - nearest hotel: SureStay Plus Hotel by Best Western Susanville
talcahuano - nearest hotel: Club Recreativo Armada de Chile
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
bayside - nearest hotel: Hotel Arcata
pismo beach - nearest hotel: Valentina Suites
lompoc - nearest hotel: Red Roof Inn Lompoc
princeville - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
22,bethel,US,41.3712,-73.4140,88,Hampton Inn Danbury
28,smithers,CA,54.7804,-127.1743,81,Sunshine Inn Hotel
62,east london,ZA,-33.0153,27.9116,59,No hotel found
94,brookings,US,44.3114,-96.7984,91,Quality Inn
130,colonia del sacramento,UY,-34.4667,-57.8500,70,Radisson Hotel Colonia del Sacramento
133,alice springs,AU,-23.7000,133.8833,81,Aurora Alice Springs
175,susanville,US,40.4163,-120.6530,36,SureStay Plus Hotel by Best Western Susanville
210,talcahuano,CL,-36.7167,-73.1167,93,Club Recreativo Armada de Chile
220,fortuna,US,40.5982,-124.1573,100,Comfort Inn & Suites Redwood Country
330,bayside,US,40.8424,-124.0637,100,Hotel Arcata


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)